In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=e1f7ce82d5e799b10b4c101d25ac91c0d09b52a5318b5b89a32dadcaa1ee17b0
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
Note: you may need to restart the kernel to use updated packages.


In [3]:
!mkdir -p vncorenlp/models/wordsegmenter

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar

!mv VnCoreNLP-1.1.1.jar vncorenlp/



!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr

!mv vi-vocab vncorenlp/models/wordsegmenter/

!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/





!mkdir -p vncorenlp/models/postagger

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/postagger/vi-tagger

!mv vi-tagger vncorenlp/models/postagger/





!mkdir -p vncorenlp/models/ner

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-500brownclusters.xz

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-ner.xz

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-pretrainedembeddings.xz

!mv vi-500brownclusters.xz vncorenlp/models/ner/

!mv vi-ner.xz vncorenlp/models/ner/

!mv vi-pretrainedembeddings.xz vncorenlp/models/ner/

--2024-12-12 14:45:37--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2024-12-12 14:45:38 (242 MB/s) - 'VnCoreNLP-1.1.1.jar' saved [27412575/27412575]

--2024-12-12 14:45:40--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting respon

In [4]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install keras-preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
VNCORE_NLP = False

max_len = 50

In [7]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

In [8]:
def tokennize_vn(text):

    sentences = vncorenlp.tokenize(text)

    s = ''

    for t in sentences:

        s = s + ' '.join(t) + ' '

    return s

In [9]:
train_dataset = '/kaggle/input/vithsd-original/train.xlsx'

dev_dataset = '/kaggle/input/vithsd-original/dev.xlsx'

test_dataset = '/kaggle/input/vithsd-original/test.xlsx'

In [10]:
import pandas as pd



train = pd.read_csv(train_dataset)

dev = pd.read_csv(dev_dataset)

test = pd.read_csv(test_dataset)

In [11]:
import itertools



aspect = list(train.columns.values[2:])

sent = [1,2,3]



itertools.product([aspect, sent])



combined_label = [a for a in itertools.product(aspect, sent)]

In [12]:
import numpy as np

import pandas as pd



def make_data(data):

    content = []

    label = []



    for i in range(0, len(data)):

        row = data.iloc[i]

        if VNCORE_NLP:

            content.append(tokennize_vn(str(row['content'])))

        else:

            content.append(row['content'])

        default_lab = np.zeros(len(combined_label), dtype=int)

        for j in range(0, len(combined_label)):

            l = combined_label[j]

            if row[l[0]] == l[1]:

                default_lab[j] = 1

        label.append(default_lab)



    return pd.DataFrame({

        'Content': content,

        'Labels': label

    })

In [13]:
train_data = make_data(train)

dev_data = make_data(dev)

test_data = make_data(test)

In [14]:
train_data['Content'] = train_data['Content'].astype(str)
dev_data['Content'] = dev_data['Content'].astype(str)
test_data['Content'] = test_data['Content'].astype(str)


In [15]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.text import Tokenizer



tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data['Content'])

vocab_size = len(tokenizer.word_index) + 1



def encoding(X, y):

    X = tokenizer.texts_to_sequences(X.values.astype(str))

    X = pad_sequences(X, maxlen=max_len, padding='post')

    y = np.asarray([np.asarray(row, dtype=float) for row in y], dtype=int)

    return (X,y)

In [16]:
train_features = encoding(train_data['Content'], train_data['Labels'])

dev_features = encoding(dev_data['Content'], dev_data['Labels'])

test_features = encoding(test_data['Content'], test_data['Labels'])

In [17]:
# Evaluation metric

import sys

import os

import os.path

from scipy.stats import sem

import numpy as np

from ast import literal_eval

import tensorflow as tf



def em_score(y_true, y_pred):

    MR = np.all(y_pred == y_true, axis=1).mean()

    return MR



def accuracy_score(y_true, y_pred):

    temp = 0

    for i in range(0, len(y_true)):

        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    
    return temp / len(y_true)


def f1_score(y_true, y_pred):

    temp = 0

    for i in range(len(y_true)):

        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):

            continue

        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i]))) / (sum(y_true[i])+sum(y_pred[i]))

    return temp/ len(y_true)

def recall(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_true[i]) == 0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_true[i])
    return temp / len(y_pred)

def precision(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_pred[i]) ==0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_pred[i])
    return temp / len(y_pred)

In [18]:
from transformers import AutoModel

#Thêm model mà yêu cầu
model = AutoModel.from_pretrained('vinai/phobert-base-v2')
embedding_matrix = model.embeddings.word_embeddings.weight.detach().numpy()
dim = embedding_matrix.shape[1]
vocab = embedding_matrix.shape[0]


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import tensorflow as tf

from tensorflow import keras



from keras.models import Sequential, Model



from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate

from keras.layers import Flatten, LSTM, Input

from tensorflow.keras.layers import Embedding



units = 100


input = Input(shape = (max_len,))

emb = Embedding(vocab, dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)

x1 = SpatialDropout1D(0.2)(emb)



x = Bidirectional(GRU(units, return_sequences = True))(x1)

x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)



y = Bidirectional(LSTM(units, return_sequences = True))(x1)

y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)



avg_pool1 = GlobalAveragePooling1D()(x)

max_pool1 = GlobalMaxPooling1D()(x)



avg_pool2 = GlobalAveragePooling1D()(y)

max_pool2 = GlobalMaxPooling1D()(y)





x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

out = Dense(15, activation = "sigmoid")(x)



model = Model(input, out)



model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 768)   │ 49,152,768 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d   │ (None, 50, 768)   │          0 │ embedding[0][0]   │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 50, 200)   │    522,000 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 50, 200)   │    695,200 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 49, 50)    │     20,050 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 49, 50)    │     20,050 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d_1[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 200)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 15)        │      3,015 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 50,413,083 (192.31 MB)

 Trainable params: 50,413,083 (192.31 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
# from tensorflow.keras.callbacks import EarlyStopping



# early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]),

          batch_size=128, epochs=20, verbose=True)

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 10s 54ms/step - acc: 0.4189 - loss: 0.3044 - val_acc: 0.5237 - val_loss: 0.2186
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.5347 - loss: 0.2155 - val_acc: 0.6012 - val_loss: 0.1976
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.6189 - loss: 0.1797 - val_acc: 0.5828 - val_loss: 0.1856
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.6670 - loss: 0.1503 - val_acc: 0.5570 - val_loss: 0.1927
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.6882 - loss: 0.1270 - val_acc: 0.5437 - val_loss: 0.1989
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.7315 - loss: 0.1002 - val_acc: 0.5445 - val_loss: 0.2238
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - acc: 0.7550 - loss: 0.0804 - val_acc: 0.5187 - val_loss: 0.2507
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.7699 - loss: 0.0641 - val_acc: 0.4696 - val_loss: 0.2759
Epoch 9/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - acc: 0.768

In [21]:
y_test = model.predict(test_features[0])

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [22]:
y_test_pred_new = []



for y in y_test:

    lb = []

    for i in range(0, len(y)):

        if y[i] >= 0.5:

            lb.append(1)

        else:

            lb.append(0)

    y_test_pred_new.append(lb)

#Aspect
new_pred = []

new_y = []

for k in range(len(y_test_pred_new)):
    result = np.sum(np.array(y_test_pred_new[k]).reshape(-1,3),axis = 1)
    new_pred.append(result)


for k in range(len(test_features[1])):
    result = np.sum(test_features[1][k].reshape(-1,3),axis = 1)
    new_y.append(result)

In [23]:
print("Aspect + Polarity\n")

print("F1 = {}".format(f1_score(test_features[1], y_test_pred_new) * 100))

print("Precision = {}".format(precision(test_features[1], y_test_pred_new)* 100))

print("Recall = {}".format(recall(test_features[1], y_test_pred_new)* 100))

Aspect + Polarity

F1 = 39.11177248677241
Precision = 41.238888888888894
Recall = 40.324074074074076


In [24]:
print("Aspect\n")

print("F1 = {}".format(f1_score(new_y, new_pred) * 100))

print("Precision = {}".format(precision(new_y, new_pred) * 100))

print("Recall = {}".format(recall(new_y, new_pred) * 100))

Aspect

F1 = 56.922883597883136
Precision = 60.54166666666667
Recall = 58.532407407407405
